# Simple Agentic RAG from Scratch

In this notebook, we'll build a simple agentic RAG system from scratch using:
- **Ollama** for local LLM inference
- **Tool calling** to give the agent capabilities
- **ReAct loop** for reasoning and acting until task completion

## What is ReAct?

ReAct (Reasoning + Acting) is a pattern where:
1. Agent **thinks** about what to do next
2. Agent **acts** by calling a tool
3. Agent **observes** the result
4. Repeat until task is complete

## Architecture

```
User Query → Agent (LLM) → Tool Call → Tool Execution → Back to Agent → Final Answer
              ↑                                            |
              |______________ ReAct Loop _________________|
```

## Installation

In [ ]:
%pip install ollama pypdf2 -q

## Step 1: Build the Tools

We'll create simple tools for working with PDFs:

In [1]:
import os
import subprocess
import glob
from pathlib import Path
from PyPDF2 import PdfReader
import re

In [2]:
def find_pdf_files(directory="./assets-resources/pdfs"):
    """
    Find all PDF files in the specified directory.

    Args:
        directory: Path to search for PDFs

    Returns:
        String with newline-separated list of PDF file paths
    """
    try:
        pdf_files = glob.glob(f"{directory}/**/*.pdf", recursive=True)
        if not pdf_files:
            return f"No PDF files found in {directory}"
        return "\n".join(pdf_files)
    except Exception as e:
        return f"Error finding PDF files: {str(e)}"

In [3]:
def search_pdf(pdf_path, search_pattern, context_lines=3):
    """
    Search for a pattern in a PDF by first converting it to text, then searching.
    
    This function combines pdf_to_text and search_in_text into one simple tool.
    
    Args:
        pdf_path: Path to the PDF file
        search_pattern: The text pattern to search for
        context_lines: Number of lines of context to show around matches (default: 3)
    
    Returns:
        String with search results and context, or error message
    """
    try:
        # Step 1: Convert PDF to text
        output_dir = "text_files"
        os.makedirs(output_dir, exist_ok=True)
        
        base_name = os.path.splitext(os.path.basename(pdf_path))[0]
        text_file_path = os.path.join(output_dir, f"{base_name}.txt")
        
        # Convert PDF to text using pdftotext
        result = subprocess.run(
            ["pdftotext", "-layout", pdf_path, text_file_path],
            capture_output=True,
            text=True,
            check=True
        )
        
        # Step 2: Search in the converted text file
        search_result = subprocess.run(
            ["grep", "-i", "-C", str(context_lines), search_pattern, text_file_path],
            capture_output=True,
            text=True
        )
        
        if search_result.returncode == 0:
            return f"Found matches for '{search_pattern}' in {pdf_path}:\n\n{search_result.stdout}"
        elif search_result.returncode == 1:
            return f"No matches found for '{search_pattern}' in {pdf_path}"
        else:
            return f"Error searching: {search_result.stderr}"
            
    except subprocess.CalledProcessError as e:
        return f"Error converting PDF: {e.stderr}"
    except FileNotFoundError:
        return "Error: pdftotext not found. Please install poppler-utils (brew install poppler on Mac)"
    except Exception as e:
        return f"Error: {str(e)}"
    

def read_full_pdf(pdf_path):
    """
    Convert a PDF file to text using pdftotext and return the full text content.

    Args:
        pdf_path: Path to the PDF file

    Returns:
        The full text content of the PDF, or an error message
    """
    import os
    import subprocess

    try:
        output_dir = "text_files"
        os.makedirs(output_dir, exist_ok=True)
        base_name = os.path.splitext(os.path.basename(pdf_path))[0]
        text_file_path = os.path.join(output_dir, f"{base_name}.txt")

        # Convert PDF to text using pdftotext
        subprocess.run(
            ["pdftotext", "-layout", pdf_path, text_file_path],
            capture_output=True,
            text=True,
            check=True
        )

        # Read and return the contents of the resulting text file
        with open(text_file_path, 'r', encoding='utf-8', errors='ignore') as f:
            return f.read()
    except subprocess.CalledProcessError as e:
        return f"Error converting PDF: {e.stderr}"
    except FileNotFoundError:
        return "Error: pdftotext not found. Please install poppler-utils (brew install poppler on Mac)"
    except Exception as e:
        return f"Error: {str(e)}"

In [4]:
def read_text_file(file_path, num_lines=None):
    """
    Read and return the content of a text file.

    Args:
        file_path: Path to the text file
        num_lines: Optional number of lines to read from the beginning

    Returns:
        The file content or error message
    """
    try:
        if num_lines:
            result = subprocess.run(
                ["head", "-n", str(num_lines), file_path],
                capture_output=True,
                text=True,
                check=True
            )
            return result.stdout
        else:
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                return f.read()
    except Exception as e:
        return f"Error reading file: {str(e)}"

### Test the tools

In [5]:
# Test: Find PDFs
pdfs = find_pdf_files()
print(f"Found PDFs:")
print(pdfs)

Found PDFs:
./assets-resources/pdfs/instruction-tune-llama2-extended-guide.pdf
./assets-resources/pdfs/lora-paper.pdf
./assets-resources/pdfs/sparks-agi-paper.pdf
./assets-resources/pdfs/qlora-paper.pdf


## Step 2: Define Tool Schemas for Ollama

We need to define the tools in a format that Ollama understands:

In [ ]:
TOOLS = [
    { 
        'type': 'function',
        'function': {
            'name': 'find_pdf_files',
            'description': 'Finds all PDF files in the specified directory and subdirectories. Use this first to discover what PDFs are available.',
            'parameters': {
                'type': 'object',
                'properties': { 
                    'directory': {
                        'type': 'string',
                        'description': 'The directory path to search for PDF files (default: ./assets-resources/pdfs)'
                    }
                },
                'required': []
            }
        }
    },
    {
        'type': 'function',
        'function': {
            'name': 'search_pdf',
            'description': 'Search for a keyword or phrase in a PDF file. This tool automatically converts the PDF to text and searches it. Returns matching excerpts with context.',
            'parameters': {
                'type': 'object',
                'properties': {
                    'pdf_path': {
                        'type': 'string',
                        'description': 'Path to the PDF file to search'
                    },
                    'search_pattern': {
                        'type': 'string',
                        'description': 'The keyword or phrase to search for'
                    },
                    'context_lines': {
                        'type': 'integer',
                        'description': 'Number of lines of context to show around matches (default: 3)'
                    }
                },
                'required': ['pdf_path', 'search_pattern']
            }
        }
    },
    {
        'type': 'function',
        'function': {
            'name': 'read_full_pdf',
            'description': 'Read the full text content of a PDF file.',
            'parameters': {
                'type': 'object',
                'properties': {
                    'pdf_path': {
                        'type': 'string',
                        'description': 'Path to the PDF file'
                    }
                },
                'required': ['pdf_path']
            }
        }
    }   
]

print("Tool schemas created successfully!")
print(f"Available tools: {', '.join([tool['function']['name'] for tool in TOOLS])}")

Tool schemas created successfully!
Available tools: find_pdf_files, search_pdf, read_full_pdf


## Step 3: Build the Simple ReAct Agent

Now we'll create a simple agent class that implements the ReAct loop:

In [8]:
import ollama
from typing import List, Dict, Any

class SimpleAgent:
    def __init__(self, model="mistral-small3.2", max_turns=10, verbose=True):
        """
        Initialize a simple ReAct agent.
        
        Args:
            model: Ollama model to use
            max_turns: Maximum number of reasoning-action turns
            verbose: Whether to print agent's reasoning process
        """
        self.model = model
        self.max_turns = max_turns
        self.verbose = verbose
        self.tools_map = {
            'find_pdf_files': find_pdf_files,
            'search_pdf': search_pdf,
            'read_text_file': read_text_file
        }
        
    def execute_tool(self, tool_name: str, arguments: Dict[str, Any]) -> str:
        """Execute a tool and return its result."""
        if tool_name not in self.tools_map:
            return f"Error: Tool '{tool_name}' not found"
        
        try:
            result = self.tools_map[tool_name](**arguments)
            return str(result)
        except Exception as e:
            return f"Error executing {tool_name}: {str(e)}"
    
    def run(self, user_query: str) -> str:
        """
        Run the agent with a user query using the ReAct loop.
        
        Args:
            user_query: The user's question or task
        
        Returns:
            Final answer from the agent
        """
        # Initialize conversation history
        messages = [
            {
                'role': 'system',
                'content': '''You are a helpful assistant that can search through PDF documents to answer questions.

Use the available tools to:
1. First, find what PDFs are available using find_pdf_files(directory="./assets-resources/pdfs")
2. Search for relevant keywords in PDFs using search_pdf(pdf_path, search_pattern, context_lines=3)
3. Read the full text content of a PDF file using read_full_pdf(pdf_path) when you need to read the entire content of a PDF file to answer the question.

Think carefully and use tools as needed. 
When you have enough information to answer the user's question, provide a clear final answer.
'''
            },
            {'role': 'user', 'content': user_query}
        ]
        
        if self.verbose:
            print(f"\n{'='*60}")
            print(f"User Query: {user_query}")
            print(f"{'='*60}\n")
        
        # ReAct loop
        for turn in range(self.max_turns):
            if self.verbose:
                print(f"\n--- Turn {turn + 1}/{self.max_turns} ---")
            
            # Get response from LLM
            response = ollama.chat(
                model=self.model,
                messages=messages,
                tools=TOOLS
            )
            
            assistant_message = response['message']
            
            # Check if agent wants to call a tool
            if 'tool_calls' in assistant_message and assistant_message['tool_calls']:
                # Add assistant's tool call to history
                messages.append(assistant_message)
                
                # Execute each tool call
                for tool_call in assistant_message['tool_calls']:
                    tool_name = tool_call['function']['name']
                    arguments = tool_call['function']['arguments']
                    
                    if self.verbose:
                        print(f"\n🔧 Tool Call: {tool_name}")
                        print(f"   Arguments: {arguments}")
                    
                    # Execute the tool
                    tool_result = self.execute_tool(tool_name, arguments)
                    
                    if self.verbose:
                        print(f"   Result: {tool_result[:200]}..." if len(tool_result) > 200 else f"   Result: {tool_result}")
                    
                    # Add tool result to messages
                    messages.append({
                        'role': 'tool',
                        'content': tool_result
                    })
            
            # Check if agent provided a final answer (no tool calls)
            elif 'content' in assistant_message:
                final_answer = assistant_message['content']
                
                if self.verbose:
                    print(f"\n✅ Final Answer (after {turn + 1} turns):")
                    print(f"\n{final_answer}")
                    print(f"\n{'='*60}\n")
                
                return final_answer
        
        # Max turns reached
        return "Max turns reached. Unable to complete the task."

## Step 4: Use the Agent

Let's create an agent and test it with some questions:

In [9]:
# Create the agent
agent = SimpleAgent(
    model="mistral-small3.2",
    max_turns=10,
    verbose=True
)

print("Agent created successfully!")

Agent created successfully!


### Example 1: Simple Question

In [11]:
answer = agent.run("What PDFs are available and what topics do they cover?")
print("\nFinal Answer:")
print(answer)


User Query: What PDFs are available and what topics do they cover?


--- Turn 1/10 ---

🔧 Tool Call: find_pdf_files
   Arguments: {'directory': './assets-resources/pdfs'}
   Result: ./assets-resources/pdfs/instruction-tune-llama2-extended-guide.pdf
./assets-resources/pdfs/lora-paper.pdf
./assets-resources/pdfs/sparks-agi-paper.pdf
./assets-resources/pdfs/qlora-paper.pdf

--- Turn 2/10 ---

✅ Final Answer (after 2 turns):

Here are the available PDFs and their topics:

1. **instruction-tune-llama2-extended-guide.pdf**: This document likely covers an extended guide on instruction tuning for Llama 2, a large language model.
2. **lora-paper.pdf**: This PDF is probably a research paper discussing LoRA (Low-Rank Adaptation), a technique used in machine learning for efficient model fine-tuning.
3. **sparks-agi-paper.pdf**: This document is likely a research paper exploring the concept of Artificial General Intelligence (AGI) and its implications.
4. **qlora-paper.pdf**: This PDF is probably 

### Example 2: Search for Specific Information

In [12]:
answer = agent.run("What is the lora technique?")
print("\nFinal Answer:")
print(answer)


User Query: What is the lora technique?


--- Turn 1/10 ---

🔧 Tool Call: find_pdf_files
   Arguments: {'directory': './assets-resources/pdfs'}
   Result: ./assets-resources/pdfs/instruction-tune-llama2-extended-guide.pdf
./assets-resources/pdfs/lora-paper.pdf
./assets-resources/pdfs/sparks-agi-paper.pdf
./assets-resources/pdfs/qlora-paper.pdf

--- Turn 2/10 ---

🔧 Tool Call: search_pdf
   Arguments: {'pdf_path': './assets-resources/pdfs/lora-paper.pdf', 'search_pattern': 'lora technique'}
   Result: No matches found for 'lora technique' in ./assets-resources/pdfs/lora-paper.pdf

--- Turn 3/10 ---

🔧 Tool Call: search_pdf
   Arguments: {'pdf_path': './assets-resources/pdfs/lora-paper.pdf', 'search_pattern': 'lora'}
   Result: Found matches for 'lora' in ./assets-resources/pdfs/lora-paper.pdf:

                                                   we pre-train larger models, full fine-tuning, which retrains all model parameter...

--- Turn 4/10 ---

✅ Final Answer (after 4 turns):

The pr

### Example 3: Multi-Step Query

In [ ]:
answer = agent.run(
    "Summarize in one sentence each of the pdfs."
)
print("\nFinal Answer:")
print(answer)


User Query: Summarize in one sentence each of the pdfs.


--- Turn 1/10 ---

🔧 Tool Call: find_pdf_files
   Arguments: {'directory': './assets-resources/pdfs'}
   Result: ./assets-resources/pdfs/instruction-tune-llama2-extended-guide.pdf
./assets-resources/pdfs/lora-paper.pdf
./assets-resources/pdfs/sparks-agi-paper.pdf
./assets-resources/pdfs/qlora-paper.pdf

--- Turn 2/10 ---

🔧 Tool Call: search_pdf
   Arguments: {'pdf_path': './assets-resources/pdfs/instruction-tune-llama2-extended-guide.pdf', 'search_pattern': 'summary'}
   Result: Found matches for 'summary' in ./assets-resources/pdfs/instruction-tune-llama2-extended-guide.pdf:

                                                                        cream.

                    ...

--- Turn 3/10 ---

🔧 Tool Call: search_pdf
   Arguments: {'pdf_path': './assets-resources/pdfs/lora-paper.pdf', 'search_pattern': 'summary'}
   Result: Found matches for 'summary' in ./assets-resources/pdfs/lora-paper.pdf:

reading comprehension (MRC)

## Understanding the ReAct Loop

Let's break down what happens in each turn:

1. **Turn 1**: Agent receives user query, decides to call `find_pdfs` to discover available documents
2. **Turn 2**: Agent receives list of PDFs, decides to call `search_pdf` with a relevant keyword
3. **Turn 3**: Agent receives search results, may search more PDFs or read specific pages
4. **Turn N**: Agent has enough information, provides final answer

The agent autonomously decides:
- Which tools to use
- What arguments to pass
- When it has enough information to answer

This is the power of agentic systems!

## Customization Tips

You can easily extend this agent by:

1. **Adding more tools**: Create new functions and add them to `TOOLS` and `tools_map`
2. **Changing the model**: Use different Ollama models like `qwen3`, etc.
3. **Adjusting max_turns**: Control how many reasoning steps the agent can take
4. **Modifying the system prompt**: Change the agent's behavior and personality
5. **Adding memory**: Store conversation history across multiple runs

## Comparison: Simple Agent vs Framework

**Our Simple Agent (~100 lines):**
- ✅ Full control over every step
- ✅ Easy to understand and debug
- ✅ No external dependencies (just Ollama)
- ✅ Perfect for learning and teaching
- ❌ Limited features
- ❌ Manual tool integration

**Framework (e.g., SmoLAgents, LangChain):**
- ✅ Many built-in tools and features
- ✅ Production-ready
- ✅ Advanced capabilities (memory, planning, etc.)
- ❌ Steeper learning curve
- ❌ Abstractions can hide important details
- ❌ Additional dependencies

**When to use each:**
- Use simple agent: Learning, prototyping, simple tasks, full control
- Use framework: Production systems, complex workflows, team projects

## Exercises

Try these challenges to deepen your understanding:

1. **Add a new tool** that can extract metadata from PDFs (author, title, creation date)
2. **Modify the system prompt** to make the agent more concise or more detailed
3. **Add conversation memory** so the agent remembers previous queries
4. **Create a comparison tool** that can search multiple PDFs and compare results
5. **Add error handling** to gracefully handle missing PDFs or invalid queries
6. **Implement streaming** to show the agent's reasoning in real-time
7. **Add a summarization tool** that can summarize entire PDFs or sections

## Conclusion

You've built a simple but powerful agentic RAG system from scratch! Key takeaways:

1. **ReAct Loop**: The pattern of Reason → Act → Observe is fundamental to agents
2. **Tool Calling**: LLMs can decide which tools to use and when
3. **Local Models**: Ollama makes it easy to run agents completely locally
4. **Simplicity**: You don't need complex frameworks to build useful agents

This foundational understanding will help you work with any agent framework and build more sophisticated systems.